# INSTALL & IMPORTS

In [12]:
#pip install selenium -q
#pip install requests -q
# pip install bs4 -q 

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

# TESTING

In [42]:
# Inicializar WebDriver
driver = webdriver.Chrome()

# Abrir la página de Bumble
driver.get("https://fr1.bumble.com/app")

In [81]:
page_source = driver.page_source
profile = BeautifulSoup(page_source, 'html.parser')
name = profile.find("span", class_='encounters-story-profile__name').text
age = profile.find("span", class_= "encounters-story-profile__age").text
about = profile.find("ul", class_= "encounters-story-about__badges").text if profile.find("ul", class_= "encounters-story-about__badges") else ""
if about:
    about = re.split('(?=[A-Z])', about)

location = profile.find("span", class_="header-2 text-color-black").text
more_info = profile.find("p", class_="encounters-story-about__text").text
job = profile.find("p", class_="encounters-story-profile__occupation").text
education = profile.find("p", class_="encounters-story-profile__education").text


print(name, age, about, location, more_info, job, education)

Joan , 30 ['174 cm', 'Active', 'Aries', 'Graduate degree', 'Socially', 'Never', 'Man', 'Something casual', 'Open to kids', 'Agnostic'] Barcelona Buscant còmplice per tast de cerveses

That's what she said Advocat Universitat Pompeu Fabra 2015


In [83]:
education

'Universitat Pompeu Fabra 2015'

In [54]:
# Esperar a que el botón esté presente y sea clickeable
try:
    button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, '.encounters-action__icon [data-qa-icon-name="floating-action-no"]'))
    )
    button.click()
    print("Botón clickeado con éxito")
except Exception as e:
    print("No se pudo clickear el botón:", e)


Botón clickeado con éxito


# FUNCTION TO TAKE ALL INFO

In [18]:
# Inicializar WebDriver
driver = webdriver.Chrome()

# Abrir la página de Bumble
driver.get("https://fr1.bumble.com/app")

In [19]:
def extract_profile_info():

    profiles = []
    prevJob = ""
    prevStories = ""
    prevEducation = ""
    
    for i in range(300):
        
        # Obtener el contenido de la página actual
        page_source = driver.page_source
    
        # Analizar el contenido de la página con BeautifulSoup
        profile = BeautifulSoup(page_source, 'html.parser')
    
    
        # Realizar un diccionario para que la información se vaya guardando.
        profiles_dict = {}
    
        # Esperar 15 segundos antes de extraer la información
        time.sleep(5)
    
        # Extraer información del perfil
        name = profile.find("span", class_='encounters-story-profile__name')
        profiles_dict["name"] = name.text if name else None
        
        age = profile.find("span", class_= "encounters-story-profile__age")
        profiles_dict["age"] = age.text if age else None
        
        about_text = profile.find("ul", class_= "encounters-story-about__badges")
        about_text = about_text.text if about_text else None
        about_categories = re.findall('[A-Z][^A-Z]*', about_text) # Se para las palabras cuando detecta una mayúscula. 
        profiles_dict["about"] = ','.join(about_categories)
    
        location = profile.find("span", class_="header-2 text-color-black")
        profiles_dict["location"] = location.text if location else ""
    

        story_about = profile.find("p", class_="encounters-story-about__text")
        profiles_dict["story_about"] = story_about.text if story_about and story_about.text != prevStories else None
        if story_about:
            prevStories = story_about.text

        
        job = profile.find("p", class_="encounters-story-profile__occupation")
        profiles_dict["job"] = job.text if job and job.text != prevJob else None
        if job:
            prevJob = job.text
    
        education = profile.find("p", class_="encounters-story-profile__education")
        profiles_dict["education"] = education.text if education and education.text != prevEducation else None
        if education:
            prevEducation = education.text
    
        # Imprime el diccionario para depuración
        print(profiles_dict)

        profiles.append(profiles_dict)
    
        try:
            button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, '.encounters-action__icon [data-qa-icon-name="floating-action-no"]'))
            )
            time.sleep(5)  # Añadir un temporizador adicional de 5 segundos
            button.click()
            print("Botón clickeado con éxito")
        except Exception as e:
            print("No se pudo clickear el botón:", e)
            

    return profiles

In [20]:
profile_info = extract_profile_info() 

# Make a DataFrame from a dict. 
df = pd.DataFrame(profile_info)

# Print DataFrame
df

{'name': 'Ray', 'age': ', 29', 'about': 'A menudo,Posgrado,En compañía,Socialmente,Mujer,Relación,No lo descarto,Aries,Agnosticismo', 'location': 'Barcelona', 'story_about': 'Steak', 'job': None, 'education': 'Univeristy of Miami 2018'}
Botón clickeado con éxito
{'name': 'Ana', 'age': ', 30', 'about': 'Hombre,Piscis', 'location': 'Barcelona', 'story_about': None, 'job': None, 'education': 'Diseñadora'}
Botón clickeado con éxito
{'name': 'Leo', 'age': ', 44', 'about': 'A menudo,Grado,En compañía,Nunca,Hombre,No lo sé aún,No quiero,Capricornio,De izquierdas,Agnosticismo', 'location': 'Barcelona', 'story_about': None, 'job': None, 'education': None}
Botón clickeado con éxito
{'name': 'Jordi A', 'age': ', 30', 'about': 'A menudo,Formación profesional,Nunca,Habitualmente,Hombre,Relación,Aún no lo tengo claro,Aries', 'location': 'Barcelona', 'story_about': '¿Es posible encontrar pareja?', 'job': 'Customer Service', 'education': 'A 2010'}
Botón clickeado con éxito
{'name': 'Jae', 'age': ', 25

,name,age,about,location,story_about,job,education
0,Ray,", 29","A menudo,Posgrado,En compañía,Socialmente,Muje...",Barcelona,Steak,None,Univeristy of Miami 2018
1,Ana,", 30","Hombre,Piscis",Barcelona,None,None,Diseñadora
2,Leo,", 44","A menudo,Grado,En compañía,Nunca,Hombre,No lo ...",Barcelona,None,None,None
3,Jordi A,", 30","A menudo,Formación profesional,Nunca,Habitualm...",Barcelona,¿Es posible encontrar pareja?,Customer Service,A 2010
4,Jae,", 25","A veces,Grado,En compañía,Nunca,Hombre,Relació...",Barcelona,"Slytherin, work in law, I'm a ⬇️\n\nHappy eith...",Team Lead en Fragomen,University of Sheffield 2019
...,...,...,...,...,...,...,...
295,Marc,", 28","A menudo,En compañía,Hombre,Relación,Algún día...",Barcelona,Catalanet de manual😊\nUna escapada a la Costa ...,None,None
296,Riki,", 37","A menudo,Posgrado,En compañía,Nunca,Hombre,Rel...","Manresa, Barcelona",Ig. Ricard.aspachs,Cirujano Veterinario,UAB 2011
297,Kari,", 25","A veces,Estudiando un posgrado,En compañía,Nun...",Barcelona,Paraguaya en Barcelona :)\nMe gustan los perri...,Diseñadora Grafica,None
298,Raquel,", 25","A menudo,Posgrado,En compañía,Mujer,Virgo",Barcelona,None,None,Psicología 2020


In [21]:
df.head()

,name,age,about,location,story_about,job,education
0,Ray,", 29","A menudo,Posgrado,En compañía,Socialmente,Muje...",Barcelona,Steak,None,Univeristy of Miami 2018
1,Ana,", 30","Hombre,Piscis",Barcelona,None,None,Diseñadora
2,Leo,", 44","A menudo,Grado,En compañía,Nunca,Hombre,No lo ...",Barcelona,None,None,None
3,Jordi A,", 30","A menudo,Formación profesional,Nunca,Habitualm...",Barcelona,¿Es posible encontrar pareja?,Customer Service,A 2010
4,Jae,", 25","A veces,Grado,En compañía,Nunca,Hombre,Relació...",Barcelona,"Slytherin, work in law, I'm a ⬇️\n\nHappy eith...",Team Lead en Fragomen,University of Sheffield 2019


In [ ]:
df.to_csv('df_man.csv', index=False)